source: https://eshop.tesco.com.my/groceries/en-GB/

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np

RETRIEVE CATEGORIES

In [2]:
start_time = time.time()
driver = webdriver.Chrome()
driver.switch_to_window(driver.current_window_handle)
driver.maximize_window()
driver.get('https://www.jayagrocer.com/')
# GET LINKS FOR EACH-SUB CATEGORY
category = driver.find_elements_by_css_selector('.list-group-item')
main = ['On-Sale Products','Special Events','Fresh Market','Dairy & Chilled','Food Essentials','Frozen Foods',\
        'Organics','Bakery','Snacks','Beverages','Alcohol','Beauty & Health','Household Products','Office Supplies',\
        'Baby Care','Pet Shop','Non-Halal']
wanted = ['Special Events','Fresh Market','Dairy & Chilled','Frozen Foods','Organics','Beverages','Alcohol','Non-Halal']
main_cats = []
urls = []
for stage1 in category:
    main_cats.append(stage1.find_element_by_tag_name('a').get_attribute('innerText'))
    urls.append(stage1.find_element_by_tag_name('a').get_attribute('href'))
time.sleep(1)
driver.quit()
print('*** Process Completeed ***')
print('Time used:\t',time.time()-start_time)

*** Process Completeed ***
Time used:	 29.9142062664032


In [3]:
data = pd.DataFrame({'sub_cat':main_cats, 'url':urls})
data = data.iloc[:104]
data['main_cat'] = data.sub_cat.apply(lambda x: x if x in main else 'nan')
data.mask(data=='nan', None,inplace=True)
data.main_cat.ffill(inplace=True)
data = data[data.main_cat.isin(wanted)]
data = data[data.main_cat != data.sub_cat]
data = data[['main_cat','sub_cat','url']]
print('Dataframe shape:\t',data.shape)
data.to_csv('jaya_grocer_category.csv',index=False)
print(data.main_cat.unique())

Dataframe shape:	 (44, 3)
['Special Events' 'Fresh Market' 'Dairy & Chilled' 'Frozen Foods'
 'Organics' 'Beverages' 'Alcohol' 'Non-Halal']


In [2]:
data = pd.read_csv('jaya_grocer_category.csv')

In [3]:
data.head()

,main_cat,sub_cat,url
0,Special Events,Mooncake Festival,https://www.jayagrocer.com/category?category_a...
1,Fresh Market,Vegetables,https://www.jayagrocer.com/category?category_a...
2,Fresh Market,Salad & Herbs,https://www.jayagrocer.com/category?category_a...
3,Fresh Market,Fruits,https://www.jayagrocer.com/category?category_a...
4,Fresh Market,Fresh Meat,https://www.jayagrocer.com/category?category_a...


RETRIEVE ITEMS

In [8]:
start_time = time.time()
driver = webdriver.Chrome()
driver.switch_to_window(driver.current_window_handle)
driver.maximize_window()
driver.get('https://www.jayagrocer.com/')

item_datas = []
for i, row in data.iterrows():
    driver.get(row.url)
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        if len(driver.find_elements_by_id('loadMore')) == 0:
            break;

    item_list = driver.find_elements_by_class_name('item-wrapper')

    for item in item_list:
        item_name = item.find_element_by_tag_name('a').get_attribute('title')
        item_vol = item.find_element_by_class_name('variant_name').get_attribute('innerText')
        price = item.find_element_by_class_name('variant_price').get_attribute('innerText').split(' ')
        item_price = price[1]
        try:
            old_price = price[3]
        except:
            old_price = '0.00'
        item_datas.append([row.main_cat, row.sub_cat, item_name, item_vol, item_price, old_price])
time.sleep(1)
driver.quit()
print('*** Process Completeed ***')
print('Time used:\t',time.time()-start_time)

*** Process Completeed ***
Time used:	 755.782662153244


In [9]:
item_df = pd.DataFrame(item_datas, columns=['main_cat','sub_cat','item_name','item_vol','item_price','item_old_price'])
print('Dataframe shape:\t',item_df.shape)
item_df.to_csv('jaya_grocer_database.csv',index=False)
item_df.head()

Dataframe shape:	 (3148, 6)


,main_cat,sub_cat,item_name,item_vol,item_price,item_old_price
0,Special Events,Mooncake Festival,Hong Kong MX Premium Assorted Mooncakes Gift Pack,1.35kg,285.00,0.00
1,Special Events,Mooncake Festival,Hong Kong MX Low Sugar Lotus Seed Paste Moonca...,1box,179.00,0.00
2,Special Events,Mooncake Festival,Hong Kong MX Lava Custard Mooncake Collection,1box,166.00,0.00
3,Special Events,Mooncake Festival,Hong Kong MX Lotus Seed Paste Mooncake with 2 ...,1box,154.80,0.00
4,Special Events,Mooncake Festival,Hong Kong MX White Lotus Seed Paste Mooncake w...,1box,150.00,0.00


In [10]:
item_df.sub_cat.value_counts()

Organic Food Cupboard             374
Wine                              278
Tea                               236
Vegetables                        196
Coffee                            179
Cheese                            150
Milk & Milk Powder                137
Soft Drinks                       123
Milk & Yoghurt                    122
Ice Cream                         106
Juice                              93
Chilled Juice & Drinks             93
Organic Dairy & Beverages          82
Pastry, Dessert & Ready Meal       78
Salad & Herbs                      62
Beer                               58
Fruits                             57
Food Cupboard                      57
Dairy Spreads                      49
Chilled Foods                      48
Pizza, Fries & Nuggets             45
Fresh Meat                         45
Cordials                           43
Water                              41
Deli Meat                          39
Chocolate & Nutritional Drinks     39
Organic Fres

In [13]:
item_df.item_old_price.value_counts()

0.00      3066
13.99        6
2.82         5
79.90        5
8.90         4
22.20        4
17.99        4
70.00        3
84.81        3
8.40         3
62.75        3
8.99         3
10.20        3
90.71        2
92.36        2
87.01        2
161.04       2
11.79        2
9.90         2
6.90         2
10.99        2
111.69       1
15.90        1
249.35       1
59.34        1
98.81        1
98.46        1
60.28        1
72.37        1
5.66         1
58.00        1
4.50         1
249.99       1
18.80        1
9.59         1
18.50        1
3.76         1
119.08       1
11.31        1
70.75        1
14.10        1
Name: item_old_price, dtype: int64